In [80]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [81]:
import numpy as np

def onehot(y):
  oh = np.zeros((y.shape[0], 10))
  for i in range(y.shape[0]):
    oh[i, int(y[i])]=1
  return oh

In [82]:
import numpy as np
def norm_zero_one(x_train):
  min_val = np.min(x_train)
  max_val = np.max(x_train)
  x_train_norm = (x_train-min_val)/(max_val-min_val)
  return x_train_norm

def norm_normal_dist(x_train):
  mean_val = np.mean(x_train)
  std_val = np.std(x_train)
  x_train_norm = (x_train-mean_val)/std_val
  return x_train_norm

In [83]:
from sklearn import preprocessing
import numpy as np

#enc = preprocessing.OneHotEncoder()

#enc.fit(y_train)

#y_train_oh = enc.transform(y_train).toarray()
#y_train_oh.shape
y_train_oh = onehot(y_train)
y_train_oh.shape

y_test_oh = onehot(y_test)
y_test_oh.shape

x_train_norm = x_train / 255.0
x_test_norm = x_test / 255.0

#x_train_norm = x_train_norm[:,:,:, np.newaxis] # Optional: Convert 60000x28x28 -> 60000x28x28x1
#x_test_norm = x_test_norm[:,:,:, np.newaxis] # Optional: Convert 60000x28x28 -> 60000x28x28x1

# Baseline

In [84]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import load_model, Model
import tensorflow as tf
import keras
import time 
from sklearn.metrics import accuracy_score

class LeNetCNN:
  # constructor
  def __init__(self):
    self.model = None

  # Define structure of the CNN
  def build(self, input_dim):
    input = Input(shape = input_dim) # X dau vao cua minh
    C1 = Conv2D(6, (3, 3), padding='same', activation='relu')(input)
    S2 = MaxPooling2D(pool_size=(2, 2), padding="same")(C1)
    C3 = Conv2D(16, (3, 3), padding='same', activation='relu')(S2)
    S4 = MaxPooling2D(pool_size=(2, 2), padding="same")(C3)

    flat = Flatten()(S4)
    # These like ANN
    F5 = Dense(120, activation='relu', use_bias=True)(flat)
    F6 = Dense(84, activation='relu', use_bias=True)(F5)
    output = Dense(10, activation='softmax', use_bias=True)(F6) # y~ output
    self.model = Model(input, output)
  
  # Train the model
  def train(self, x_train, y_train, x_val, y_val): # x_train chinh la X, y_train chinh la ground-truth
    sgd = keras.optimizers.SGD(learning_rate=0.01)
    adam = keras.optimizers.Adam(learning_rate=0.01)
    self.model.compile(optimizer=sgd, loss = 'categorical_crossentropy', metrics=['accuracy'])
    self.model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 5, batch_size = 128)
  
  # Load model from file
  def load(self, model_file):
    self.model = load_model(model_file)
  
  # save the trained model
  def save(self, model_file):
    self.model.save(model_file)
  
  # Show the architecture of the model
  def summary(self):
    self.model.summary()

  # Test the model with a given input
  def predict(self, x_test):
    return self.model.predict(x_test)

  def evaluate(self, x_test, y_test):
    return self.model.evaluate(x_test, y_test)

  def get_num_params(self):
    train_params = np.sum([np.prod(v.get_shape()) for v in self.model.trainable_weights])
    non_train_params = np.sum([np.prod(v.get_shape()) for v in self.model.non_trainable_weights])
    return [train_params, non_train_params]
    

In [85]:
cnn = LeNetCNN()
cnn.build((28, 28, 1))
cnn.summary()
start = time.time()
cnn.train(x_train_norm, y_train_oh, x_test_norm, y_test_oh)
cnn_time = time.time() - start 
cnn_score = cnn.evaluate(x_test_norm, y_test_oh)

Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_44 (Conv2D)          (None, 28, 28, 6)         60        
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 14, 14, 6)        0         
 g2D)                                                            
                                                                 
 conv2d_45 (Conv2D)          (None, 14, 14, 16)        880       
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 7, 7, 16)         0         
 g2D)                                                            
                                                                 
 flatten_18 (Flatten)        (None, 784)               0  

# Without Activate Function

In [86]:
class LeNetCNN_withoutActivateFunction:
  # constructor
  def __init__(self):
    self.model = None

  # Define structure of the CNN
  def build(self, input_dim):
    input = Input(shape = input_dim) # X dau vao cua minh
    C1 = Conv2D(6, (3, 3), padding='same', activation=None)(input)
    S2 = MaxPooling2D(pool_size=(2, 2), padding="same")(C1)
    C3 = Conv2D(16, (3, 3), padding='same', activation=None)(S2)
    S4 = MaxPooling2D(pool_size=(2, 2), padding="same")(C3)

    flat = Flatten()(S4)
    # These like ANN
    F5 = Dense(120, activation=None, use_bias=True)(flat)
    F6 = Dense(84, activation=None, use_bias=True)(F5)
    output = Dense(10, activation=None, use_bias=True)(F6) # y~ output
    self.model = Model(input, output)
  
  # Train the model
  def train(self, x_train, y_train, x_val, y_val): # x_train chinh la X, y_train chinh la ground-truth
    sgd = keras.optimizers.SGD(learning_rate=0.01)
    adam = keras.optimizers.Adam(learning_rate=0.01)
    self.model.compile(optimizer=sgd, loss = 'categorical_crossentropy', metrics=['accuracy'])
    self.model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 5, batch_size = 128)
  
  # Load model from file
  def load(self, model_file):
    self.model = load_model(model_file)
  
  # save the trained model
  def save(self, model_file):
    self.model.save(model_file)
  
  # Show the architecture of the model
  def summary(self):
    self.model.summary()

  # Test the model with a given input
  def predict(self, x_test):
    return self.model.predict(x_test)

  def evaluate(self, x_test, y_test):
    return self.model.evaluate(x_test, y_test)

  def get_num_params(self):
    train_params = np.sum([np.prod(v.get_shape()) for v in self.model.trainable_weights])
    non_train_params = np.sum([np.prod(v.get_shape()) for v in self.model.non_trainable_weights])
    return [train_params, non_train_params]

In [87]:
cnn_withoutActivateFunction = LeNetCNN_withoutActivateFunction()
cnn_withoutActivateFunction.build((28, 28, 1))
cnn_withoutActivateFunction.summary()
start = time.time()
cnn_withoutActivateFunction.train(x_train_norm, y_train_oh, x_test_norm, y_test_oh)
cnn_withoutActivateFunction_time = time.time() - start 
cnn_withoutActivateFunction_score = cnn_withoutActivateFunction.evaluate(x_test_norm, y_test_oh)

Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_46 (Conv2D)          (None, 28, 28, 6)         60        
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 14, 14, 6)        0         
 g2D)                                                            
                                                                 
 conv2d_47 (Conv2D)          (None, 14, 14, 16)        880       
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 7, 7, 16)         0         
 g2D)                                                            
                                                                 
 flatten_19 (Flatten)        (None, 784)               0  

# Without Pooling

In [88]:
class LeNetCNN_withoutPooling:
  # constructor
  def __init__(self):
    self.model = None

  # Define structure of the CNN
  def build(self, input_dim):
    input = Input(shape = input_dim) # X dau vao cua minh
    C1 = Conv2D(6, (3, 3), padding='same', activation='relu')(input)
    # S2 = MaxPooling2D(pool_size=(2, 2), padding="same")(C1)
    C3 = Conv2D(16, (3, 3), padding='same', activation='relu')(C1)
    # S4 = MaxPooling2D(pool_size=(2, 2), padding="same")(C3)

    flat = Flatten()(C3)
    # These like ANN
    F5 = Dense(120, activation='relu', use_bias=True)(flat)
    F6 = Dense(84, activation='relu', use_bias=True)(F5)
    output = Dense(10, activation='softmax', use_bias=True)(F6) # y~ output
    self.model = Model(input, output)
  
  # Train the model
  def train(self, x_train, y_train, x_val, y_val): # x_train chinh la X, y_train chinh la ground-truth
    sgd = keras.optimizers.SGD(learning_rate=0.01)
    adam = keras.optimizers.Adam(learning_rate=0.01)
    self.model.compile(optimizer=sgd, loss = 'categorical_crossentropy', metrics=['accuracy'])
    self.model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 5, batch_size = 128)
  
  # Load model from file
  def load(self, model_file):
    self.model = load_model(model_file)
  
  # save the trained model
  def save(self, model_file):
    self.model.save(model_file)
  
  # Show the architecture of the model
  def summary(self):
    self.model.summary()

  # Test the model with a given input
  def predict(self, x_test):
    return self.model.predict(x_test)

  def evaluate(self, x_test, y_test):
    return self.model.evaluate(x_test, y_test)

In [89]:
cnn_withoutPooling = LeNetCNN_withoutPooling()
cnn_withoutPooling.build((28, 28, 1))
cnn_withoutPooling.summary()
start = time.time()
cnn_withoutPooling.train(x_train_norm, y_train_oh, x_test_norm, y_test_oh)
cnn_withoutPooling_time = time.time() - start 
cnn_withoutPooling_score = cnn_withoutPooling.evaluate(x_test_norm, y_test_oh)

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_48 (Conv2D)          (None, 28, 28, 6)         60        
                                                                 
 conv2d_49 (Conv2D)          (None, 28, 28, 16)        880       
                                                                 
 flatten_20 (Flatten)        (None, 12544)             0         
                                                                 
 dense_58 (Dense)            (None, 120)               1505400   
                                                                 
 dense_59 (Dense)            (None, 84)                10164     
                                                                 
 dense_60 (Dense)            (None, 10)                850

# Result

In [90]:
print("Baseline: Training time: {:.2f} - Accuracy: {:.2f}".format(cnn_time, cnn_score[1]))
print("Without Activate Function: Training time: {:.2f} - Accuracy: {:.2f}".format(cnn_withoutActivateFunction_time, cnn_withoutActivateFunction_score[1]))
print("Without Pooling: {:.2f} - Accuracy: {:.2f}".format(cnn_withoutPooling_time, cnn_withoutPooling_score[1]))

Baseline: Training time: 12.04 - Accuracy: 0.96
Without Activate Function: Training time: 11.41 - Accuracy: 0.10
Without Pooling: 11.47 - Accuracy: 0.96
